In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train', 'test']


In [32]:
import random
import gc # Gabage collector for cleaning deleted data from memory

train_dir = '../input/train/train'
test_dir = '../input/test/test'

# train_imgs = ['../input/train/{}'.format(i) for i in os.listdir(train_dir)] # get full data set
train_0s = ['../input/train/train/{}'.format(i) for i in os.listdir(train_dir) if i.startswith("0_")] # get dog images
train_1s = ['../input/train/train/{}'.format(i) for i in os.listdir(train_dir) if i.startswith("1_")] # get cat images

test_imgs = ['../input/test/test/{}'.format(i) for i in os.listdir(test_dir)] # get test images

train_imgs = train_0s[:2000] + train_1s[:2000]  # slice the dataset and use 2000 in each class
random.shuffle(train_imgs)  # shuffle it randomly
print(len(train_imgs))
print(len(test_imgs))

1777
184


In [33]:
import cv2
import spacy

In [34]:
nrows = 256
ncols = 256
channels = 3

In [35]:
def read_and_resize_image(list_of_images):
    X = []
    y = []
    for image in list_of_images:
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        try:
            X.append(cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), (nrows,ncols), interpolation=cv2.INTER_CUBIC))
        except:
            print(image)
            continue
        if (image.split('/'))[-1].startswith("0_"):
            y.append(0)
        elif (image.split('/'))[-1].startswith("1_"):
            y.append(1)        
    return X, y

In [36]:
X, y = read_and_resize_image(train_imgs)
X = np.array(X)
y = np.array(y)

../input/train/train/0_1559025172_eb490af59a.jpg


In [37]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=2)

In [38]:
print("Shape of X train: ", X_train.shape)
print("Shape of X Val: ", X_val.shape)
print("Shape of y train: ", y_train.shape)
print("Shape of y val: ", y_val.shape)

ntrain = len(X_train)
nval = len(X_val)

batch_size = 32

Shape of X train:  (1420, 256, 256, 3)
Shape of X Val:  (356, 256, 256, 3)
Shape of y train:  (1420,)
Shape of y val:  (356,)


In [39]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, BatchNormalization
from keras import backend as k 

In [40]:
from keras.applications import VGG19

In [41]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [42]:
for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [43]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 8, 8, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_10 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_11 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 129       
Total params: 20,615,489
Trainable params: 591,105
Non-trainable params: 20,024,384
__________________________________________________________

In [45]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=5e-6), metrics=['acc'])

In [46]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img

train_datagen = ImageDataGenerator(rescale=1./255, # Scale the image between 0 and 1
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

In [47]:
train_generator = train_datagen.flow(X_train, y_train,batch_size=batch_size, shuffle=True)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

In [49]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=ntrain // batch_size,
                              epochs=75,
                              validation_data=val_generator,
                              validation_steps=nval // batch_size)

Epoch 1/75
44/44 [==============================] - 20s 454ms/step - loss: 0.6890 - acc: 0.5668 - val_loss: 0.6881 - val_acc: 0.5463
Epoch 2/75
44/44 [==============================] - 18s 404ms/step - loss: 0.6887 - acc: 0.5663 - val_loss: 0.6888 - val_acc: 0.5741
Epoch 3/75
11/44 [======>.......................] - ETA: 11s - loss: 0.6882 - acc: 0.5369

KeyboardInterrupt: 

In [ ]:
model_json = model.to_json()
with open("flickr_favs_arch.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("flickr_favs_weights.h5")